# Расчет результатов

1. Посчитать модели линейной регрессии для 20 зданий по оптимальному набору параметров: метеорологические данные, дни недели, недели года, месяцы и праздники по всему набору данных.
2. Загрузить данные решения, посчитать значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузить результат в файл.

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных 20 зданий из HDF5

In [2]:
energy = pd.read_hdf("../data/out/energy.0-20.h5", "energy")
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               175680 non-null  datetime64[ns]
 1   building_id             175680 non-null  int8          
 2   meter_reading           175680 non-null  float16       
 3   primary_use             175680 non-null  category      
 4   air_temperature         175680 non-null  float16       
 5   cloud_coverage          175680 non-null  float16       
 6   dew_temperature         175680 non-null  float16       
 7   precip_depth_1_hr       175680 non-null  float16       
 8   sea_level_pressure      175680 non-null  float16       
 9   wind_direction          175680 non-null  float16       
 10  wind_speed              175680 non-null  float16       
 11  air_temperature_diff_1  175680 non-null  float16       
 12  air_temperature_diff_2  175680

## Загрузка данных и оптимизация памяти

In [3]:
buidings = pd.read_csv("../data/buildings.csv", usecols=["site_id", "building_id"])

weather = pd.read_csv("../data/weather_test.csv.gz")
weather = weather[weather["site_id"] == 0]
weather = weather.drop(columns=["wind_direction"], axis=1)

results = pd.read_csv("../data/test.csv.gz")
results = results[(results["building_id"] < 20) & (results["meter"] == 0)]
results = pd.merge(
    left=results,
    right=buidings,
    how="left",
    left_on="building_id",
    right_on="building_id",
)

del buidings

results = results.drop(columns=["meter"], axis=1)
print(results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350400 entries, 0 to 350399
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   row_id       350400 non-null  int64 
 1   building_id  350400 non-null  int64 
 2   timestamp    350400 non-null  object
 3   site_id      350400 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 13.4+ MB
None


## Интерполяция значений и обогащение погодных данных для одного года

In [5]:
interpolate_columns = [
    "air_temperature",
    "dew_temperature",
    "cloud_coverage",
    "wind_speed",
    "sea_level_pressure",
]

for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction="both", kind="cubic")

weather["air_temperature_diff_1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff_1"] = weather.at[1, "air_temperature_diff_1"]
weather["air_temperature_diff_2"] = weather["air_temperature_diff_1"].diff()
weather.at[0, "air_temperature_diff_2"] = weather.at[1, "air_temperature_diff_2"]

## Объединение данных по погоде

In [6]:
results = results.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])

results = pd.merge(
    left=results,
    right=weather,
    how="left",
    left_index=True,
    right_index=True,
)

results.reset_index(inplace=True)
results = results.drop(columns=["site_id"], axis=1)

del weather

results = reduce_mem_usage(results)
print(results.info())

Потребление памяти меньше на - 19.72 Мб (минус 67.0%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350400 entries, 0 to 350399
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               350400 non-null  datetime64[ns]
 1   row_id                  350400 non-null  int32         
 2   building_id             350400 non-null  int8          
 3   air_temperature         350400 non-null  float16       
 4   cloud_coverage          350400 non-null  float16       
 5   dew_temperature         350400 non-null  float16       
 6   precip_depth_1_hr       349800 non-null  float16       
 7   sea_level_pressure      350400 non-null  float16       
 8   wind_speed              350400 non-null  float16       
 9   air_temperature_diff_1  350400 non-null  float16       
 10  air_temperature_diff_2  350400 non-null  float16       
dtypes: datetime64[ns](1), float16(8), int

## Обогащение данных по дате

In [ ]:
results["hour"] = results["timestamp"].dt.hour.astype("int8")
results["weekday"] =results["timestamp"].dt.weekday.astype("int8")
results["week"] =results["timestamp"].dt.isocalendar().week.astype("int8")
results["month"] =results["timestamp"].dt.month.astype("int8")
results["date"] = pd.to_datetime(results["timestamp"].dt.date)

dates_range = pd.date_range(start="2016-12-31", end="2018-06-01")
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
results["is_holiday"] = results["date"].isin(us_holidays).astype("int8")

for weekday in range(0, 7):
    results[f"is_wday {str(weekday)}"] = results["weekday"].isin([weekday]).astype("int8")
for week in range(1, 54):
    results[f"is_w {str(week)}"] = results["week"].isin([week]).astype("int8")
for month in range(1, 13):
    results[f"is_m {str(month)}"] = results["month"].isin([month]).astype("int8")